In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1246309,2021-05-05,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1246310,2021-05-06,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1246311,2021-05-07,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1246312,2021-05-08,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
33247,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
33249,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
33251,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
33253,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
33255,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
766550,2021-05-05,Arkansas,Arkansas,5001,2096,36.00,5000,Arkansas,AR,Arkansas,State,3017804
766552,2021-05-06,Arkansas,Arkansas,5001,2097,36.00,5000,Arkansas,AR,Arkansas,State,3017804
766554,2021-05-07,Arkansas,Arkansas,5001,2098,36.00,5000,Arkansas,AR,Arkansas,State,3017804
766556,2021-05-08,Arkansas,Arkansas,5001,2101,36.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1246309,2021-05-05,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1246310,2021-05-06,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1246311,2021-05-07,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1246312,2021-05-08,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-05-09') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
474,2021-05-09,Snohomish,Washington,37023,571.00,53061,WA,County,822083,4503.56,69.46
946,2021-05-09,Cook,Illinois,542450,10611.00,17031,IL,County,5150233,10532.53,206.03
1417,2021-05-09,Orange,California,270744,5008.00,6059,CA,County,3175692,8525.51,157.70
1887,2021-05-09,Maricopa,Arizona,541555,9922.00,4013,AZ,County,4485414,12073.69,221.21
2357,2021-05-09,Los Angeles,California,1236316,24022.00,6037,CA,County,10039107,12315.00,239.28
...,...,...,...,...,...,...,...,...,...,...,...
1245601,2021-05-09,Wheeler,Oregon,28,1.00,41069,OR,County,1332,2102.10,75.08
1245810,2021-05-09,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1245988,2021-05-09,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00
1246162,2021-05-09,Loving,Texas,2,0.00,48301,TX,County,169,1183.43,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
474,2021-05-09,Snohomish,Washington,37023,571.00,53061,WA,County,822083,4503.56,69.46,69.46,4503.56
946,2021-05-09,Cook,Illinois,542450,10611.00,17031,IL,County,5150233,10532.53,206.03,206.03,10532.53
1417,2021-05-09,Orange,California,270744,5008.00,6059,CA,County,3175692,8525.51,157.70,157.70,8525.51
1887,2021-05-09,Maricopa,Arizona,541555,9922.00,4013,AZ,County,4485414,12073.69,221.21,221.21,12073.69
2357,2021-05-09,Los Angeles,California,1236316,24022.00,6037,CA,County,10039107,12315.00,239.28,239.28,12315.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245601,2021-05-09,Wheeler,Oregon,28,1.00,41069,OR,County,1332,2102.10,75.08,75.08,2102.10
1245810,2021-05-09,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1245988,2021-05-09,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00,0.00,4352.81
1246162,2021-05-09,Loving,Texas,2,0.00,48301,TX,County,169,1183.43,0.00,0.00,1183.43


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
801557,2021-05-09,Hale,Alabama,2229,77.00,1065,AL,County,14651,15213.98,525.56,525.56,15213.98,1
872527,2021-05-09,Clarke,Alabama,3510,61.00,1025,AL,County,23622,14859.03,258.23,258.23,14859.03,2
627081,2021-05-09,Lowndes,Alabama,1388,53.00,1085,AL,County,9726,14271.03,544.93,544.93,14271.03,3
466506,2021-05-09,Franklin,Alabama,4257,82.00,1059,AL,County,31362,13573.75,261.46,261.46,13573.75,4
555984,2021-05-09,Etowah,Alabama,13831,353.00,1055,AL,County,102268,13524.27,345.17,345.17,13524.27,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1181085,2021-05-09,Platte,Wyoming,624,11.00,56031,WY,County,8393,7434.77,131.06,131.06,7434.77,19
764929,2021-05-09,Converse,Wyoming,1001,17.00,56009,WY,County,13822,7242.08,122.99,122.99,7242.08,20
951114,2021-05-09,Lincoln,Wyoming,1405,12.00,56023,WY,County,19830,7085.22,60.51,60.51,7085.22,21
1058660,2021-05-09,Niobrara,Wyoming,161,2.00,56027,WY,County,2356,6833.62,84.89,84.89,6833.62,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
627081,2021-05-09,Lowndes,Alabama,1388,53.00,1085,AL,County,9726,14271.03,544.93,544.93,14271.03,3,1
801557,2021-05-09,Hale,Alabama,2229,77.00,1065,AL,County,14651,15213.98,525.56,525.56,15213.98,1,2
266611,2021-05-09,Walker,Alabama,7235,277.00,1127,AL,County,63521,11389.93,436.08,436.08,11389.93,27,3
554340,2021-05-09,Clay,Alabama,1562,56.00,1027,AL,County,13235,11802.04,423.12,423.12,11802.04,19,4
676278,2021-05-09,Greene,Alabama,923,34.00,1063,AL,County,8111,11379.61,419.18,419.18,11379.61,28,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
674641,2021-05-09,Hot Springs,Wyoming,369,3.00,56017,WY,County,4413,8361.66,67.98,67.98,8361.66,17,19
927396,2021-05-09,Uinta,Wyoming,2217,13.00,56041,WY,County,20226,10961.14,64.27,64.27,10961.14,5,20
951114,2021-05-09,Lincoln,Wyoming,1405,12.00,56023,WY,County,19830,7085.22,60.51,60.51,7085.22,21,21
265360,2021-05-09,Teton,Wyoming,3751,9.00,56039,WY,County,23464,15986.19,38.36,38.36,15986.19,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,69.46,4503.56,19,24
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,69.46,4503.56,19,24
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,69.46,4503.56,19,24
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,69.46,4503.56,19,24
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,69.46,4503.56,19,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1243390,2021-05-05,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1243391,2021-05-06,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1243392,2021-05-07,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1243393,2021-05-08,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
798233,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,525.56,15213.98,2,1,1.00
798234,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,525.56,15213.98,2,1,0.00
798235,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,525.56,15213.98,2,1,0.00
798236,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,525.56,15213.98,2,1,0.00
798237,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,525.56,15213.98,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
967038,2021-05-05,Crook,Wyoming,429,12.00,56011,WY,County,7584,5656.65,158.23,158.23,5656.65,6,23,0.00
967039,2021-05-06,Crook,Wyoming,429,12.00,56011,WY,County,7584,5656.65,158.23,158.23,5656.65,6,23,0.00
967040,2021-05-07,Crook,Wyoming,429,12.00,56011,WY,County,7584,5656.65,158.23,158.23,5656.65,6,23,0.00
967041,2021-05-08,Crook,Wyoming,429,12.00,56011,WY,County,7584,5656.65,158.23,158.23,5656.65,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
624569,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14271.03,1,3,1.00,0.00
624570,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14271.03,1,3,0.00,0.00
624571,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14271.03,1,3,0.00,0.00
624572,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14271.03,1,3,0.00,0.00
624573,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14271.03,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
622514,2021-05-05,Albany,Wyoming,4285,12.00,56001,WY,County,38880,11021.09,30.86,30.86,11134.26,23,4,14.00,0.00
622515,2021-05-06,Albany,Wyoming,4302,12.00,56001,WY,County,38880,11064.81,30.86,30.86,11134.26,23,4,17.00,0.00
622516,2021-05-07,Albany,Wyoming,4329,12.00,56001,WY,County,38880,11134.26,30.86,30.86,11134.26,23,4,27.00,0.00
622517,2021-05-08,Albany,Wyoming,4329,12.00,56001,WY,County,38880,11134.26,30.86,30.86,11134.26,23,4,0.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-05-09') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
318287,2021-05-09,Imperial,California,28373,721.00,6025,CA,County,181215,15657.09,397.87,397.87,15657.09,1,2,21.00,0.00
2357,2021-05-09,Los Angeles,California,1236316,24022.00,6037,CA,County,10039107,12315.00,239.28,239.28,12315.00,2,5,229.00,14.00
158451,2021-05-09,San Bernardino,California,296596,4707.00,6071,CA,County,2180085,13604.79,215.91,215.91,13604.79,3,4,82.00,10.00
557580,2021-05-09,Inyo,California,1424,38.00,6027,CA,County,18039,7894.01,210.65,210.65,7894.01,4,24,0.00,0.00
70837,2021-05-09,Stanislaus,California,61916,1055.00,6099,CA,County,550660,11243.96,191.59,191.59,11243.96,5,9,0.00,0.00
32817,2021-05-09,Riverside,California,299064,4587.00,6065,CA,County,2470546,12105.18,185.67,185.67,12105.18,6,7,0.00,0.00
56355,2021-05-09,San Joaquin,California,73009,1391.00,6077,CA,County,762148,9579.37,182.51,182.51,9579.37,7,18,0.00,0.00
87834,2021-05-09,Tulare,California,49554,839.00,6107,CA,County,466195,10629.46,179.97,179.97,10629.46,8,10,0.00,0.00
31959,2021-05-09,Fresno,California,101676,1682.00,6019,CA,County,999101,10176.75,168.35,168.35,10176.75,9,14,51.00,-2.00
505273,2021-05-09,Merced,California,31743,458.00,6047,CA,County,277680,11431.50,164.94,164.94,11431.50,10,8,0.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1180636,2021-05-09,Lassen,California,5698,24.00,6035,CA,County,30573,18637.36,78.50,78.50,18637.36,34,1,0.00,0.00
318287,2021-05-09,Imperial,California,28373,721.00,6025,CA,County,181215,15657.09,397.87,397.87,15657.09,1,2,21.00,0.00
676220,2021-05-09,Kings,California,22971,246.00,6031,CA,County,152940,15019.62,160.85,160.85,15019.62,11,3,6.00,0.00
158451,2021-05-09,San Bernardino,California,296596,4707.00,6071,CA,County,2180085,13604.79,215.91,215.91,13604.79,3,4,82.00,10.00
2357,2021-05-09,Los Angeles,California,1236316,24022.00,6037,CA,County,10039107,12315.00,239.28,239.28,12315.00,2,5,229.00,14.00
199233,2021-05-09,Kern,California,109031,1373.00,6029,CA,County,900202,12111.84,152.52,152.52,12111.84,14,6,38.00,0.00
32817,2021-05-09,Riverside,California,299064,4587.00,6065,CA,County,2470546,12105.18,185.67,185.67,12105.18,6,7,0.00,0.00
505273,2021-05-09,Merced,California,31743,458.00,6047,CA,County,277680,11431.50,164.94,164.94,11431.50,10,8,0.00,0.00
70837,2021-05-09,Stanislaus,California,61916,1055.00,6099,CA,County,550660,11243.96,191.59,191.59,11243.96,5,9,0.00,0.00
87834,2021-05-09,Tulare,California,49554,839.00,6107,CA,County,466195,10629.46,179.97,179.97,10629.46,8,10,0.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
318287,2021-05-09,Imperial,California,28373,721.00,6025,CA,County,181215,15657.09,397.87,397.87,15657.09,1,2,21.00,0.00
2357,2021-05-09,Los Angeles,California,1236316,24022.00,6037,CA,County,10039107,12315.00,239.28,239.28,12315.00,2,5,229.00,14.00
158451,2021-05-09,San Bernardino,California,296596,4707.00,6071,CA,County,2180085,13604.79,215.91,215.91,13604.79,3,4,82.00,10.00
557580,2021-05-09,Inyo,California,1424,38.00,6027,CA,County,18039,7894.01,210.65,210.65,7894.01,4,24,0.00,0.00
70837,2021-05-09,Stanislaus,California,61916,1055.00,6099,CA,County,550660,11243.96,191.59,191.59,11243.96,5,9,0.00,0.00
32817,2021-05-09,Riverside,California,299064,4587.00,6065,CA,County,2470546,12105.18,185.67,185.67,12105.18,6,7,0.00,0.00
56355,2021-05-09,San Joaquin,California,73009,1391.00,6077,CA,County,762148,9579.37,182.51,182.51,9579.37,7,18,0.00,0.00
87834,2021-05-09,Tulare,California,49554,839.00,6107,CA,County,466195,10629.46,179.97,179.97,10629.46,8,10,0.00,0.00
31959,2021-05-09,Fresno,California,101676,1682.00,6019,CA,County,999101,10176.75,168.35,168.35,10176.75,9,14,51.00,-2.00
505273,2021-05-09,Merced,California,31743,458.00,6047,CA,County,277680,11431.50,164.94,164.94,11431.50,10,8,0.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,397.87,15657.09,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,397.87,15657.09,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,397.87,15657.09,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,397.87,15657.09,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,397.87,15657.09,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4172,05/05/21,Lassen,5693,24.00,18621.01,78.50,78.50,18637.36,34,1,1.00,0.00
4173,05/06/21,Lassen,5696,24.00,18630.82,78.50,78.50,18637.36,34,1,3.00,0.00
4174,05/07/21,Lassen,5697,24.00,18634.09,78.50,78.50,18637.36,34,1,1.00,0.00
4175,05/08/21,Lassen,5698,24.00,18637.36,78.50,78.50,18637.36,34,1,1.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,397.87,15657.09,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,397.87,15657.09,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,397.87,15657.09,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,397.87,15657.09,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,397.87,15657.09,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4258,05/05/21,Merced,31701,454.00,11416.38,163.50,164.94,11431.50,10,8,19.00,0.00
4259,05/06/21,Merced,31711,456.00,11419.98,164.22,164.94,11431.50,10,8,10.00,2.00
4260,05/07/21,Merced,31743,458.00,11431.50,164.94,164.94,11431.50,10,8,32.00,2.00
4261,05/08/21,Merced,31743,458.00,11431.50,164.94,164.94,11431.50,10,8,0.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)